In [3]:
#pip install SpeechRecognition
# !pip install pyaudio
# !pip install nltk
# !pip install fsspec>=2023.5.0
# !pip install pyttsx3


                                              0.0/184.3 kB ? eta -:--:--
     ------------------------------------- 184.3/184.3 kB 10.9 MB/s eta 0:00:00


In [2]:
#RECONHECIMENTO E TRANSCRIÇÃO
#USANDO ARQUIVO EXTERNO

import speech_recognition as sr

def transcrever_audio(caminho_audio):
    recognizer = sr.Recognizer()

    with sr.AudioFile(caminho_audio) as audio_file:
        audio = recognizer.record(audio_file)

        try:
            texto_transcrito = recognizer.recognize_google(audio, language="en") #language = "en" para inglês, "pt-BR" para português
            return texto_transcrito
        except sr.UnknownValueError:
            print("Não foi possível reconhecer o áudio")
        except sr.RequestError as e:
            print(f"Erro na requisição para o serviço de reconhecimento de fala: {e}")
            
#arquivos disponíveis no repositório:
#Audio 1 - en - reconhecimento.wav - áudio em inglês
#Audio 1 - reconhecimento.wav - áudio em português
#Gravando.wav - áudio em inglês

caminho_do_audio = "audios teste reconhecimento/Gravando.wav"
texto_transcrito = transcrever_audio(caminho_do_audio)

if texto_transcrito:
    print("Texto transcrito:")
    print(texto_transcrito)


Texto transcrito:
shit I left my keys at home


In [14]:
#ANÁLISE COM PNL

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# nltk.download('vader_lexicon')

def analisar_sentimento(texto):
    sia = SentimentIntensityAnalyzer()
    resultado = sia.polarity_scores(texto)

    if resultado['compound'] >= 0.05:
        return 'Positivo'
    elif resultado['compound'] <= -0.05:
        return 'Negativo'
    else:
        return 'Neutro'

if __name__ == "__main__":
    sentimento = analisar_sentimento(texto_transcrito) #texto_transcrito deve ser a variável gerada no passo anterior

    print(f"Sentimento: {sentimento}")


Sentimento: Negativo


In [ ]:
#TENTATIVA DE INTERFACE

import speech_recognition as sr
import tkinter as tk
from threading import Thread

class TranscricaoAudioGUI:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Transcrição de Áudio")

        self.label_status = tk.Label(self.root, text="Pressione Iniciar para falar.")
        self.label_status.pack(pady=10)

        self.botao_iniciar = tk.Button(self.root, text="Iniciar", command=self.iniciar_transcricao)
        self.botao_iniciar.pack(pady=10)

        self.botao_parar = tk.Button(self.root, text="Parar", command=self.parar_transcricao, state=tk.DISABLED)
        self.botao_parar.pack(pady=10)

        self.transcricao_texto = tk.Text(self.root, height=5, width=40)
        self.transcricao_texto.pack(pady=10)

        self.recognizer = sr.Recognizer()
        self.microfone = sr.Microphone()

        self.transcricao_thread = None
        self.transcricao_em_andamento = False

    def iniciar_transcricao(self):
        self.label_status.config(text="Gravação em andamento...")
        self.botao_iniciar.config(state=tk.DISABLED)
        self.botao_parar.config(state=tk.NORMAL)
        self.transcricao_texto.delete(1.0, tk.END)

        self.transcricao_em_andamento = True
        self.transcricao_thread = Thread(target=self.transcrever_audio_microfone)
        self.transcricao_thread.start()

    def parar_transcricao(self):
        self.label_status.config(text="Pressione Iniciar para falar.")
        self.botao_iniciar.config(state=tk.NORMAL)
        self.botao_parar.config(state=tk.DISABLED)
        self.transcricao_em_andamento = False

    def transcrever_audio_microfone(self):
        while self.transcricao_em_andamento:
            with self.microfone as source:
                print("Fale algo...")
                self.recognizer.adjust_for_ambient_noise(source)
                audio = self.recognizer.listen(source, timeout=10)

            try:
                texto_transcrito = self.recognizer.recognize_google(audio, language="pt-BR")
                self.transcricao_texto.insert(tk.END, texto_transcrito + "\n")
            except sr.UnknownValueError:
                print("Não foi possível reconhecer o áudio")
            except sr.RequestError as e:
                print(f"Erro na requisição para o serviço de reconhecimento de fala: {e}")

    def iniciar_aplicacao(self):
        self.root.mainloop()

if __name__ == "__main__":
    aplicacao = TranscricaoAudioGUI()
    aplicacao.iniciar_aplicacao()


Fale algo...
